# Gravitational Lensing

AstroPhot is now part of the caskade ecosystem. caskade simulators can interface
very easily since the parameter management is handled automatically. Here we
demonstrate how the caustics package, which is also written in caskade, can be
used to add gravitational lensing to AstroPhot models. This is similar to the
Custom Models tutorial although more specific.

In [ ]:
import astrophot as ap
import matplotlib.pyplot as plt
import caustics
import numpy as np
import torch
import socket

socket.setdefaulttimeout(120)

In [ ]:
class LensSersic(ap.models.SersicGalaxy):
    _model_type = "lensed"

    def __init__(self, *args, lens, **kwargs):
        super().__init__(*args, **kwargs)
        self.lens = lens

    def transform_coordinates(self, x, y):
        x, y = self.lens.raytrace(x, y)
        x, y = super().transform_coordinates(x, y)
        return x, y

In [ ]:
target = ap.TargetImage(
    filename="https://www.legacysurvey.org/viewer/fits-cutout?ra=177.1380&dec=19.5008&size=150&layer=ls-dr9&pixscale=0.262&bands=g",
    name="horseshoe",
    variance="auto",
    zeropoint=22.5,
)
target.psf = target.psf_image(data=ap.utils.initialize.gaussian_psf(0.5, 51, 0.262))

cosmology = caustics.FlatLambdaCDM(name="cosmology")
lens = caustics.SIE(
    name="lens",
    x0=0.28,
    y0=0.79,
    q=0.9,
    phi=2.5 * np.pi / 10,
    Rein=5.5,
    z_l=0.4457,
    z_s=2.379,
    cosmology=cosmology,
)
lens.to_dynamic()
lens.z_l.to_static()
lens.z_s.to_static()
source = ap.Model(
    name="source",
    model_type="lensed sersic galaxy model",
    lens=lens,
    center=[0.2, 0.42],
    q=0.6,
    PA=np.pi / 3,
    n=1,
    Re=0.1,
    Ie=1.5,
    target=target,
    psf_convolve=True,
)
lenslight = ap.Model(
    name="lenslight",
    model_type="sersic galaxy model",
    center=lambda p: torch.stack((p.x0.value, p.y0.value)),
    q=lens.q,
    PA=0,
    n=4.7,
    Re=1,
    Ie=0.2,
    target=target,
    psf_convolve=True,
)
lenslight.center.link((lens.x0, lens.y0))

model = ap.Model(
    name="horseshoe",
    model_type="group model",
    models=[source, lenslight],
    target=target,
)
model.initialize()

fig, axarr = plt.subplots(1, 3, figsize=(15, 4))
ap.plots.target_image(fig, axarr[0], target)
axarr[0].set_title("Target Image")
ap.plots.model_image(fig, axarr[1], model)
axarr[1].set_title("Model Image")
ap.plots.residual_image(fig, axarr[2], model)
axarr[2].set_title("Residual Image")
plt.show()

Note that we give reasonable starting parameters for the lensing model. Gravitational lensing is notoriously hard to model, so we need to start near the correct minimum otherwise we may easily fall to some poor local minimum.

In [ ]:
model.graphviz()

In [ ]:
res = ap.fit.LM(model, verbose=1).fit()

In [ ]:
fig, axarr = plt.subplots(1, 3, figsize=(15, 4))
ap.plots.target_image(fig, axarr[0], target)
axarr[0].set_title("Target Image")
ap.plots.model_image(fig, axarr[1], model, vmax=32)
axarr[1].set_title("Model Image")
ap.plots.residual_image(fig, axarr[2], model)
axarr[2].set_title("Residual Image")
plt.show()

This is not an exceptionally good fit, but it is well known that the horseshoe requires a more detailed model than an SIE lens. The cool result here is that we were able to link AstroPhot and caustics very easily to create a detailed lensing model!